In [6]:
import pandas as pd
import json

In [7]:
df_redefine = pd.read_csv('./raw_data/ariel_project_data_1Kdays.csv')
df_tagged = pd.read_json('./raw_data/new_etherscan_data.json')
print(df_redefine.head())
df_tagged.head()

   Unnamed: 0                               token_address project_name  \
0           0  0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0        Matic   
1           1  0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2        Maker   
2           2  0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32         Lido   
3           3  0xD533a949740bb3306d119CC777fa900bA034cd52        Curve   
4           4  0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9         Aave   

                                   code_repo  \
0  https://github.com/maticnetwork/contracts   
1         https://github.com/makerdao/dai.js   
2    https://github.com/lidofinance/lido-dao   
3  https://github.com/curvefi/curve-contract   
4      https://github.com/aave/aave-protocol   

                                token_creator     total_tvl  \
0  0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC  6.303445e+09   
1  0x731c6F8C754FA404CFCC2Ed8035EF79262f65702  6.018573e+09   
2  0x909d05F384D0663eD4BE59863815aB43b4f347Ec  5.927611e+09   
3  0xc4AD0Ef33

,0xd970227827a64abe0243879edcd5a56d386f3640,0xd7dcd9b99787c619b4d57979521258d1a7267ad7,0x3e3a4c795b0c4ca93ac21d8e4af6144fed09e3e8,0xa43ea2d76d346b9655fd266af14429225e5a7e23,0x8efae5a67f3d1805ac592b95afb21cb3a7f54cb5,0xe0673ad293b32a86634c38f65402bd5f22d2aade,0x752b3f956a157ba0d613aca1c8be4bda8207d200,0x87f5e8c3425218837f3cb67db941af0c01323e56,0x4824a7b64e3966b0133f4f4ffb1b9d6beb75fff7,0xa3c084ae80a3f03963017669bc696e961d3ae5d5,...,0x303fbc94757f75e2e3ee255beb1c6e0e2464a658,0x5687b9ba2300f5822d265e1865ebad36e7dac856,0xa55a0baaab6741d446986322751b8cc8484c6c81,0xa5a4cccccaa26cea096f6e493839423f4d66c63f,0xc5817a4c5e8ec6488c9a26c6862ff3060757b498,0xa5dc252b96de88d9f4dcb5f687e5c0d049389aff,0x7fc95945eaa14e7a2954052a4c9bfbaa79d170ae,0x39246c4f3f6592c974ebc44f80ba6dc69b817c71,0x45cde5c8ae6e4a5568e488ef43a173eeeabfcc21,0xf82b70fe5a41c0d87ae4939de2fc0ecbb13ff5a0
name,Multichain Hack Alert 2565,EvrynetToken (EVRY),,,Bancor Network: bnUOS Token,,Balancer: cDAI/DAI 50/50 #2,BitCoin ONE (BTCONE),TokenClub To... (TCT),Uleypool,...,Balancer: ETH/RAI/FRY 25/25/50,,Balancer: ETH/DMG 10/90 #5,Synth sBRL (sBRL),SushiSwap: MAYC,Zerion: Bancor Adapter,Balancer: AUC/ETH 80/20 #4,Opyn: Options Exchange,DOOS (DOOS),SushiSwap: cDAI
labels,[take-action],[finance],[rocket-pool],[airdrop-hunter],[bancor],[blocked],[balancer],[website-down],[investment],[mining],...,[balancer],[airdrop-hunter],[balancer],[deprecated],[sushiswap],[zerion],[balancer],[opyn],[website-down],[sushiswap]


In [8]:
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: x.replace("'",'"'))
df_redefine['other_contracts'] = df_redefine['other_contracts'].apply(lambda x: json.loads(x))


In [9]:
def parse_contracts(obj):
    x=[]
    for address in obj:
        y = address['smartContract']['address']['address']
        x.append(y)
    return x

df_redefine['other_created_address'] = df_redefine['other_contracts'].apply(lambda x: parse_contracts(x))

In [10]:
df_redefine.columns

Index(['Unnamed: 0', 'token_address', 'project_name', 'code_repo',
       'token_creator', 'total_tvl', 'other_contracts',
       'other_created_address'],
      dtype='object')

In [11]:
df_redefine_expanded = df_redefine.loc[:,['token_creator','other_created_address','project_name']]
df_redefine_expanded

,token_creator,other_created_address,project_name
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,"[0xb83e7f1b67d9e57faaa82452ddea2ec9db54f2e8, 0...",Matic
1,0x731c6F8C754FA404CFCC2Ed8035EF79262f65702,[],Maker
2,0x909d05F384D0663eD4BE59863815aB43b4f347Ec,[],Lido
3,0xc4AD0Ef33A0A4ddA3461c479ccb6c36d1e4B7Be4,"[0x679fcb9b33fc4ae10ff4f96caef49c1ae3f8fa67, 0...",Curve
4,0x51F22ac850D29C879367A77D241734AcB276B815,"[0xcd55262c3ea354a58661597f71037d5fa26b72bd, 0...",Aave
...,...,...,...
457,0x09E9FF67d9D5A25Fa465Db6f0bEdE5560581f8Cb,"[0x9c90e89573fa85e24b0236de44fbf4046dc8dcc9, 0...",Spice (SFI)
458,0x94627695F66Ab36Ae00c1995a30Bf5B30E139873,"[0xcd19bc5bfb5b98f19711e9d26e6b6d6460724b31, 0...",Dracula Token
459,0x70217E7De3A68187905269462506f81cF344bbad,[],FOX (FOX)
460,0x490CE4616672e93B1c8f5E43aa80312Fd73dEE8C,"[0xbbdba695be29a043881c74d288797f50e0dc4259, 0...",ParaSwap (PSP)


In [12]:
df_redefine_expanded = df_redefine_expanded.explode('other_created_address')

In [13]:
df_redefine_expanded

,token_creator,other_created_address,project_name
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0xb83e7f1b67d9e57faaa82452ddea2ec9db54f2e8,Matic
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x4c37c126eac3baaa98e38eae05215c522e6ca817,Matic
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x185486869e615f5a5644991a491212582caa08fb,Matic
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x4a5020dc88541c4e0ed19fd7e1da2f6722012c56,Matic
0,0x78655080b65f42E2ceE5FA5673689CC44D4E1cFC,0x40730f34668afcb3884f050cbc3d376a444bbe44,Matic
...,...,...,...
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0x60c6b5dc066e33801f2d9f2830595490a3086b4e,Dogs Of Elon
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0x3c40601f73fbf50b81a72edbf2786f14ebb7371b,Dogs Of Elon
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0xbef1b61dc62b82659f102a332e1276e67371ae6e,Dogs Of Elon
461,0x8a2B212cA369f4893F298dD0C4aebE0E75C8396e,0xf8e9f10c22840b613cda05a0c5fdb59a4d6cd7ef,Dogs Of Elon


In [14]:
df_etherscan = df_tagged.transpose()
df_etherscan.reset_index(inplace=True)
df_etherscan


,index,name,labels
0,0xd970227827a64abe0243879edcd5a56d386f3640,Multichain Hack Alert 2565,[take-action]
1,0xd7dcd9b99787c619b4d57979521258d1a7267ad7,EvrynetToken (EVRY),[finance]
2,0x3e3a4c795b0c4ca93ac21d8e4af6144fed09e3e8,,[rocket-pool]
3,0xa43ea2d76d346b9655fd266af14429225e5a7e23,,[airdrop-hunter]
4,0x8efae5a67f3d1805ac592b95afb21cb3a7f54cb5,Bancor Network: bnUOS Token,[bancor]
...,...,...,...
27360,0xa5dc252b96de88d9f4dcb5f687e5c0d049389aff,Zerion: Bancor Adapter,[zerion]
27361,0x7fc95945eaa14e7a2954052a4c9bfbaa79d170ae,Balancer: AUC/ETH 80/20 #4,[balancer]
27362,0x39246c4f3f6592c974ebc44f80ba6dc69b817c71,Opyn: Options Exchange,[opyn]
27363,0x45cde5c8ae6e4a5568e488ef43a173eeeabfcc21,DOOS (DOOS),[website-down]


In [15]:
df_merge = pd.merge(df_redefine_expanded, df_etherscan, left_on="other_created_address",right_on="index", how="left")
df_is_tagged = df_merge.loc[df_merge['index'].notna()]

In [16]:
len(df_redefine_expanded.drop_duplicates('other_created_address'))
len(df_etherscan.drop_duplicates('index'))

27365

In [17]:
df_is_tagged.set_index('index', inplace=True)


In [18]:
df_is_tagged.drop('other_created_address', axis=1,inplace=True)

/var/folders/00/_mvycxm948781kjwpdqn_f1w0000gn/T/ipykernel_3665/3853981218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_is_tagged.drop('other_created_address', axis=1,inplace=True)


Data stats:

In [19]:
print("Tokens analyzed: %s"%(len(df_redefine)))
print("Num of contracts created by token creator: %s"%(len(df_redefine_expanded)))
print("Tagged contracts analyzed: %s"%(len(df_etherscan)))
print("Num of created contracts that are tagged: %s"%(len(df_merge.loc[df_merge['index'].notna()])))

Tokens analyzed: 462
Num of contracts created by token creator: 15472
Tagged contracts analyzed: 27365
Num of created contracts that are tagged: 743


As can be seen - there is very few common contracts between the Redefine DF and Etherscan tagged data. 
To overcome this we will filter out contracts that haven't been used in the past year.

Additional ways for us to increase validation of contracts:
1. Scrape Etherscan up to date V
2. Add a date threshold 
3. Call contract name + call name function

In [20]:
df_is_tagged.to_csv('first_results.csv')

We will verify from the tagged Etherscan names which are correct and which are incorrect

In [21]:
filter = df_is_tagged.drop_duplicates('project_name')

In [22]:
remove_words = ["token",
"proxy",
"contract",
"work",
"protocol",
"governance",
"dao",
"finance"]



In [23]:
def verify_name(p_name, labels,name):
    project_name = p_name.split()
    print(project_name)
    for name in project_name:
        if name in remove_words:
            project_name.remove(name)
    print(project_name)




df_is_tagged['verified'] = df_is_tagged['project_name','labels','name'].apply(lambda x: verify_name(x))

KeyError: ('project_name', 'labels', 'name')

In [ ]:
def verify_name(p_name, labels,name):
    project_name = p_name.split()
    lower_project_name = []
    for name in project_name:
        name = name.lower()
        name = name.
        lower_project_name.append(name)
        if name in remove_words:
            lower_project_name.remove(name)
    print(lower_project_name)

verify_name('Spell Token (SPELL)','','')

['spell', '(spell)']
